In [141]:
import pandas as pd

In [142]:
#importing dataset of NYC restaurant information, violations, and grades from the Dept of Mental Health and Hygiene
#https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j



#original file was upwards of 170mb, preliminary data cleaning involving removing unnecessary columns was done in Excel

df = pd.read_csv("./DOHMH_New_York_City_Restaurant_Inspection_Results.csv")
df.head()

,DBA,BORO,CUISINE DESCRIPTION,VIOLATION CODE,SCORE,GRADE,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,17.0,B,40.829178,-73.875707
1,ARIANA KEBAB HOUSE,Manhattan,Afghan,10B,5.0,A,40.764993,-73.987856
2,TAP NYC,Manhattan,Café/Coffee/Tea,08C,20.0,NaN,40.777716,-73.978556
3,GEORGES RESTAURANT,Brooklyn,American,10F,7.0,A,40.641088,-74.014474
4,Q & I BAKERY,Brooklyn,Bakery,06C,22.0,NaN,40.628482,-74.006099


How scores and grades are determined
https://www1.nyc.gov/assets/doh/downloads/pdf/rii/restaurant-grading-faq.pdf

GRADE DEFINITIONS
N = Not Yet Graded
A = Grade A
B = Grade B
C = Grade C
Z = Grade Pending
P= Grade Pending issued on re-opening following an initial inspection that resulted in a closure

In [143]:
df.shape

(397490, 8)

In [144]:
df.dtypes

DBA                     object
BORO                    object
CUISINE DESCRIPTION     object
VIOLATION CODE          object
SCORE                  float64
GRADE                   object
Latitude               float64
Longitude              float64
dtype: object

In [145]:
df.columns

Index(['DBA', 'BORO', 'CUISINE DESCRIPTION', 'VIOLATION CODE', 'SCORE',
       'GRADE', 'Latitude', 'Longitude'],
      dtype='object')

In [146]:
#renaming columns

df = df.rename(columns={"DBA":"Name","BORO":"Borough", "CUISINE DESCRIPTION":"Cuisine", "VIOLATION CODE":"Violation Code", "SCORE":"Score", "GRADE":"Grade"})

df.columns

Index(['Name', 'Borough', 'Cuisine', 'Violation Code', 'Score', 'Grade',
       'Latitude', 'Longitude'],
      dtype='object')

In [147]:
#removing rows with null values from Name, Violation Code, and Grade rows

df = df.dropna(subset=['Name', 'Violation Code', 'Grade'])
del df['Score']

df.head()

,Name,Borough,Cuisine,Violation Code,Grade,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,B,40.829178,-73.875707
1,ARIANA KEBAB HOUSE,Manhattan,Afghan,10B,A,40.764993,-73.987856
3,GEORGES RESTAURANT,Brooklyn,American,10F,A,40.641088,-74.014474
7,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,A,40.704818,-73.934015
8,"DUNKIN', BASKIN ROBBINS",Manhattan,Donuts,10B,A,40.750971,-73.996492


In [148]:
#df.shape

In [149]:
df = df[df.Borough != "Manhattan"]

#removing rows with missing Longitude and Latitude values
df = df[df.Latitude != 0]
df = df[df.Longitude != 0]

#removing rows with Grades of "N", "G", and "Z", which are unnecessary for our analysis
df = df[df.Grade != "N"]
df = df[df.Grade != "G"]
df = df[df.Grade != "Z"]

#may keep p grades for some data viz
df = df[df.Grade != "P"]

df.shape

(116273, 7)

In [150]:
df.head()

,Name,Borough,Cuisine,Violation Code,Grade,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,B,40.829178,-73.875707
3,GEORGES RESTAURANT,Brooklyn,American,10F,A,40.641088,-74.014474
7,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,A,40.704818,-73.934015
11,GOOD HOPE RESTAURANT,Queens,Chinese,08A,B,40.687063,-73.822045
15,MORNING GLORY BAKERY,Queens,Bakery,10F,A,40.728130,-73.863607


In [151]:
#importing csv of violation code descriptions and their maximum penalty
#original dataset was stored as a word doc PDF and converted to a tabular format with Excel
#https://www1.nyc.gov/assets/doh/downloads/pdf/rii/ri-violation-penalty.pdf

codedf = pd.read_csv("./ViolationCodes.csv")
codedf.head()

,Violation Code,Violation Group,Violation Description,Maximum Penalty
0,02A,2,Food temperature storage violation,600
1,02B,2,Food temperature storage violation,600
2,02C,2,Food temperature storage violation,300
3,02D,2,Food temperature storage violation,300
4,02E,2,Food temperature storage violation,200


In [152]:
#index df by violation code

codedf.set_index('Violation Code')

,Violation Group,Violation Description,Maximum Penalty
Violation Code,,,
02A,2,Food temperature storage violation,600
02B,2,Food temperature storage violation,600
02C,2,Food temperature storage violation,300
02D,2,Food temperature storage violation,300
02E,2,Food temperature storage violation,200
...,...,...,...
22B,22,Failure to take reasonable precautions to prot...,200
22C,22,Failure to take reasonable precautions to prot...,200
22E,22,Failure to take reasonable precautions to prot...,200


In [153]:
#merge restaurant data and violation code data frame on the "Violation Code" column

mergeddf = pd.merge(df, codedf, on='Violation Code')
mergeddf

,Name,Borough,Cuisine,Violation Code,Grade,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,B,40.829178,-73.875707,10,Facility and machinery related,200
1,GEORGES RESTAURANT,Brooklyn,American,10F,A,40.641088,-74.014474,10,Facility and machinery related,200
2,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,A,40.704818,-73.934015,10,Facility and machinery related,200
3,MORNING GLORY BAKERY,Queens,Bakery,10F,A,40.728130,-73.863607,10,Facility and machinery related,200
4,LA SABROSURA,Queens,"Latin (Cuban, Dominican, Puerto Rican, South &...",10F,A,40.766028,-73.919554,10,Facility and machinery related,200
...,...,...,...,...,...,...,...,...,...,...
116268,EMPANADAS MONUMENTAL,Bronx,"Latin (Cuban, Dominican, Puerto Rican, South &...",03G,C,40.854345,-73.911797,3,Food not from an approved source,300
116269,GOOD MORNING BROOKLYN,Brooklyn,Salads,03G,A,40.706331,-73.922465,3,Food not from an approved source,300
116270,THE WILD GOOSE,Queens,Irish,03G,C,40.744798,-73.910247,3,Food not from an approved source,300
116271,ANGIE'S BREAKFAST SPOT,Brooklyn,"Latin (Cuban, Dominican, Puerto Rican, South &...",03G,A,40.684938,-73.914031,3,Food not from an approved source,300


In [154]:
mergeddf.columns

Index(['Name', 'Borough', 'Cuisine', 'Violation Code', 'Grade', 'Latitude',
       'Longitude', 'Violation Group', 'Violation Description',
       'Maximum Penalty'],
      dtype='object')

In [155]:
#counts of each grade... most restaurants are in the A&B grade range!
gradecounts = df.groupby('Grade').count()
gradecounts

,Name,Borough,Cuisine,Violation Code,Latitude,Longitude
Grade,,,,,,
A,95054,95054,95054,95054,94972,94972
B,15654,15654,15654,15654,15644,15644
C,5565,5565,5565,5565,5565,5565


In [156]:
#group combined dataframe by Grades
groupedbygrade = df.groupby('Grade')

In [157]:
#group of A grade restaurants
groupA= groupedbygrade.get_group('A')
groupA.head()

,Name,Borough,Cuisine,Violation Code,Grade,Latitude,Longitude
3,GEORGES RESTAURANT,Brooklyn,American,10F,A,40.641088,-74.014474
7,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,A,40.704818,-73.934015
15,MORNING GLORY BAKERY,Queens,Bakery,10F,A,40.728130,-73.863607
18,LA SABROSURA,Queens,"Latin (Cuban, Dominican, Puerto Rican, South &...",10F,A,40.766028,-73.919554
28,FRENCH LOUIE,Brooklyn,French,10F,A,40.688170,-73.988061


In [158]:
#converting A grade group to dictionary
groupA.reset_index(inplace=True)
Adata=groupA.to_dict("records")
#Adata

In [159]:
#group of B grade restaurants

groupB= groupedbygrade.get_group('B')
groupB.head()

,Name,Borough,Cuisine,Violation Code,Grade,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,B,40.829178,-73.875707
11,GOOD HOPE RESTAURANT,Queens,Chinese,08A,B,40.687063,-73.822045
37,JR PRIMOS RESTAURANT,Bronx,Spanish,10F,B,40.865413,-73.867511
43,MOJAVE,Queens,Southwestern,04H,B,40.775332,-73.911679
55,FULL KEE SEAFOOD RESTURANT,Queens,Chinese,08A,B,40.760941,-73.830333


In [160]:
#converting B grade group to dictionary

groupB.reset_index(inplace=True)
Bdata=groupB.to_dict("records")
#Bdata

In [161]:
#group of C grade restaurants

groupC= groupedbygrade.get_group('C')
groupC.head()

,Name,Borough,Cuisine,Violation Code,Grade,Latitude,Longitude
90,AUTENTICO TACOS AL SUADERO,Queens,Mexican,05D,C,40.748151,-73.879038
131,OBAAPA AFRICAN & CARIBBEAN RESTAURANT,Bronx,African,04L,C,40.860269,-73.902719
189,EIGHT JANE FOOD,Queens,Asian,10F,C,40.761359,-73.831040
325,YOLIE'S BAR & RESTAURANT,Brooklyn,Creole,04H,C,40.644856,-73.922561
376,LA CASA DE JULIA,Queens,Mexican,04N,C,40.745965,-73.899844


In [162]:
#converting C grade group to dictionary

groupC.reset_index(inplace=True)
Cdata=groupC.to_dict("records")
#Cdata

In [163]:
#group of P grade restaurants

#groupP= groupedbygrade.get_group('P')
#groupP.head()

In [164]:
#converting P grade group to dictionary

#groupP.reset_index(inplace=True)
#Pdata=groupP.to_dict("records")
#Pdata

In [165]:
#Prepping to export dataframes to MongoDB
import pymongo

In [166]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client["restaurantgradesDB"]

#collections by grade
collectionA = db["GroupA"]
collectionB = db["GroupB"]
collectionC = db["GroupC"]
#collectionP = db["GroupP"]

In [ ]:
# Insert data from grade groups into MongoDB collection
collectionA.insert_many(Adata)

In [ ]:
collectionB.insert_many(Bdata)

In [ ]:
collectionC.insert_many(Cdata)

In [ ]:
#collectionP.insert_many(Pdata)